# Script to populate database - 1st part;

## Generate CSV

This script will generate CSV with all emails in Gmail account following the tags of subjects.

## Set external variables

In [ ]:
# number of years to get history emails;
n_years = 30

# email user;
user = ''

# list tags of email;
tags = ['GestNFe', 'StacNFe', 'StacCTe']

## Script

### Librarys

In [ ]:
import pandas as pd
import numpy as np
from simplegmail import Gmail
from simplegmail.message import Message
from simplegmail.query import construct_query

### Functions

In [ ]:
def get_name_system(row_name):
    system = row_name
    if 'StacNFe' in row_name:
        system = 'StacNFe'
    elif 'StacCTe' in row_name:
        system = 'StacCTe'
    elif 'GestNFe' in row_name:
        system = 'GestNFe'
    elif 'NFe' in row_name:
        system = 'StacNFe'
        
    return system

### Script

In [ ]:
gmail = Gmail()

messages = []
for tag in tags:
    query_params = {
        "newer_than": (n_years, 'year'),
        "subject": tag,
    }
    
    lmessages = gmail.get_messages(user_id=user, query=construct_query(query_params))
    messages.append([x.html for x in lmessages])

In [ ]:
for i in range(len(tags)):
    print('Tag {} has {} registers!'.format(tags[i], len(messages[i])))

In [ ]:
data = []
for tag_message in messages:
    for message in tag_message:
        if (message is not None) and ('Razão Social' in message):
            message_lines = message.split('\r\n')
            
            if 'Nota Fiscal' in message_lines[2]:
                tags_message =  ['Nota Fiscal', 'Serie', 'Chave Nfe', 'Data Emissão', 'Razão Social',
                                 'Nome Fanta', 'Cnpj', 'Endereço', 'Bairro', 'Cidade']                            
            elif 'NFC-e' in message_lines[2]:
                tags_message = ['NFC-e', 'Serie', 'Chave NFC-e', 'Data Emissão', 'Razão Social',
                                'Nome Fanta', 'Cnpj', 'Endereço', 'Bairro', 'Cidade']
            elif 'NF-e' in message_lines[2]:
                tags_message = ['NF-e', 'Serie', 'Chave NF-e', 'Data Emissão', 'Razão Social',
                                'Nome Fanta', 'Cnpj', 'Endereço', 'Bairro', 'Cidade']
            elif 'CTe' in message_lines[2]:
                tags_message = ['CTe', 'Serie', 'Chave CTe', 'Data Emissão', 'Razão Social',
                                'Nome Fanta', 'Cnpj', 'Endereço', 'Bairro', 'Cidade']
            else:
                print("Tags does't found, list message: {}".format(' '.join(message_lines)))
            
            data_locale = {'system': get_name_system(message_lines[0])}
            
            for tag in tags_message:
                for line in message_lines:
                    if (tag in line):
                        if ('Chave' not in line) or (('Chave' in tag) and ('Chave' in line)):
                            data_locale.update({tag: line[line.find(':')+2:len(line)]})
            
            data.append(data_locale)

In [ ]:
df_list = []
for d in data:
    values = list(d.values())
    
    if len(values) < 11:
        print('Fail in register!\nFields:\n{}\nValues:\n{}!'.format(', '.join(list(d.keys())),
                                                                 ', '.join(list(d.values()))))
        
    type_d = 'Undefined'
    if list(d.keys())[1].upper() in ['NOTA FISCAL', 'NF-E']:
        type_d = 'NF-e'
    elif list(d.keys())[1].upper() in ['NFC-E']:
        type_d = 'NFC-e'
    elif list(d.keys())[1].upper() in ['CTE']:
        type_d = 'CT-e'
        
    city = str(values[10])
    if not pd.isnull(city):
        if '</p>' in city:
            city = city[0:city.find('</p>')]
    
    key = str(values[3])
    month_year = np.nan
    if not pd.isnull(key):
        if len(key) > 0:
            if 'CTe' in key:
                key = key.replace('CTe', '')
                
            month_year = '{}/{}'.format(key[4:6], key[2:4])
    
    oficial_name, nick_name = values[5], values[6]
    if '&amp;' in oficial_name or '&amp;' in nick_name:
        oficial_name = oficial_name.replace('&amp;', '&')
        nick_name = nick_name.replace('&amp;', '&')
    
    date = str(values[4])
    if not pd.isnull(date):
        day, month, year = date.split('/')
        if len(year) < 4:
            year = '19' if int(year) >= 30 else '20' + year;
            
        if int(day) > 31 or int(month) > 12:
            aux = day
            day = month
            month = aux

        day = '0' + day if len(day) < 2 else day
        month = '0' + month if len(month) < 2 else month

        date = '/'.join([day, month, year])
        
    register = {'sistema': str(values[0]),
                'tipo_nota': type_d,
                'nf_nfc_ct': str(values[1]),
                'serie': str(values[2]),
                'chave': key,
                'data': date,
                'mes_ano': month_year,
                'razao_social': oficial_name,
                'nome_fantasia': nick_name,
                'CNPJ': str(values[7]),
                'endereco': str(values[8]),
                'bairro': str(values[9]),
                'cidade': city}
    
    df_list.append(register)
    
df = pd.DataFrame(df_list)

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
print('N rows original: {}'.format(df.shape[0]))

df_duplicate = df[df.duplicated()]
print('N rows duplicate: {}'.format(df_duplicate.shape[0]))

df = df.drop_duplicates()
print('N rows after drop duplicates: {}'.format(df.shape[0]))

In [ ]:
df.to_csv('emails_informations.csv', index=False)

In [ ]:
df.shape